In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import feature_selection_helpers as fsh

In [2]:
# Read in NSCH data
df = pd.read_sas('data/nsch_2019e_topical.sas7bdat')

In [3]:
# Use feature selection helpers to make series containing the correlation and histogram-overlap scores
corr_series = fsh.make_corr_series(df)
hist_series = fsh.make_overlap_series(df)

/Users/gyoung19/Library/CloudStorage/OneDrive-KennesawStateUniversity/Python/Erdos/Health_Edu_Project/feature_selection_helpers.py:35: RuntimeWarning: invalid value encountered in divide
  return (hist1.astype(np.float32) / hist1.sum(), hist2.astype(np.float32) / hist2.sum())
/Users/gyoung19/Library/CloudStorage/OneDrive-KennesawStateUniversity/Python/Erdos/Health_Edu_Project/feature_selection_helpers.py:35: RuntimeWarning: invalid value encountered in divide
  return (hist1.astype(np.float32) / hist1.sum(), hist2.astype(np.float32) / hist2.sum())
/Users/gyoung19/Library/CloudStorage/OneDrive-KennesawStateUniversity/Python/Erdos/Health_Edu_Project/feature_selection_helpers.py:35: RuntimeWarning: invalid value encountered in divide
  return (hist1.astype(np.float32) / hist1.sum(), hist2.astype(np.float32) / hist2.sum())
/Users/gyoung19/Library/CloudStorage/OneDrive-KennesawStateUniversity/Python/Erdos/Health_Edu_Project/feature_selection_helpers.py:35: RuntimeWarning: invalid value enco

In [4]:
#Define some thresholds defining which features to keep based on metric
corr_thresh = 0.15
hist_thresh = 0.8

#Refine the series with only the elements that meet the above thresholds
corr_series_keep = corr_series[corr_series > corr_thresh]
hist_series_keep = hist_series[hist_series < hist_thresh]

In [5]:
# Define sets containing the features to keep according to the series returned above
corr_keep_features = set(corr_series_keep.index)
hist_keep_features = set(hist_series_keep.index)
#Define their union
union_features = corr_keep_features.union(hist_keep_features)

In [6]:
#Some info about the features
print('There are ' + str(len(corr_keep_features)) + ' features in corr_keep_features and ' + str(len(hist_keep_features)) + ' features in hist_keep_features.')
print('There are ' + str(len(corr_keep_features.difference(hist_keep_features))) + ' features in corr_keep_features that are not in hist_keep_features.')
print('There are ' + str(len(hist_keep_features.difference(corr_keep_features))) + ' features in hist_keep_features that are not in corr_keep_features.')
print('Their intersection contains ' + str(len(hist_keep_features.intersection(corr_keep_features))) + ' features and their union contains ' +
       str(len(hist_keep_features.union(corr_keep_features))) + ' features.')

There are 34 features in corr_keep_features and 36 features in hist_keep_features.
There are 18 features in corr_keep_features that are not in hist_keep_features.
There are 20 features in hist_keep_features that are not in corr_keep_features.
Their intersection contains 16 features and their union contains 54 features.


In [7]:
# Read in NSCH dctionary of feature we already plan to use
NSCH_dict = pd.read_csv('data/NSCH_dictionary.csv')

In [8]:
# Set of handpicked features
handpicked_features = set(NSCH_dict['col_name_2019'].values)

In [9]:
#Some info about the features
print('There are ' + str(len(union_features)) + ' features in union_features and ' + str(len(handpicked_features)) + ' features in handpicked_features.')
print('Their intersection contains ' + str(len(union_features.intersection(handpicked_features))) + ' features and their union contains ' +
       str(len(union_features.union(handpicked_features))) + ' features.')
print('Including the features found using the correlation and histogram analysis will add ' + str(len(union_features.difference(handpicked_features)))
       + ' features to our model.')

There are 54 features in union_features and 53 features in handpicked_features.
Their intersection contains 14 features and their union contains 93 features.
Including the features found using the correlation and histogram analysis will add 40 features to our model.


In [10]:
# Get the features that were identified by the correlation/historgram analysis that we did not include in our handpicked variables
new_features = list(union_features.difference(handpicked_features))

In [11]:
# Look at a dataframe only including these new features
df_new = df[new_features]
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29433 entries, 0 to 29432
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FWC             29433 non-null  float64
 1   ARTHRITIS_DESC  81 non-null     float64
 2   K2Q46C          170 non-null    float64
 3   SC_K2Q22        29396 non-null  float64
 4   DOWNSYN_DESC    43 non-null     float64
 5   K5Q10           29297 non-null  float64
 6   SESPLANMO       418 non-null    float64
 7   K7Q84_R         20976 non-null  float64
 8   TOTCSHCN        29433 non-null  float64
 9   SC_K2Q13        29394 non-null  float64
 10  K2Q33C          2926 non-null   float64
 11  ARTHRITIS_CURR  125 non-null    float64
 12  HEADACHE_DESC   960 non-null    float64
 13  K7Q04R_R        21021 non-null  float64
 14  LIVEUSA_MO      53 non-null     float64
 15  K2Q32C          1251 non-null   float64
 16  DECISIONS       25955 non-null  float64
 17  K7Q85_R         21077 non-null 

In [12]:
#Many of the features are missing >90% of their data. Let's get rid of columns with sufficiently high proportions of null
df_trimmed = df_new.loc[:, df_new.isnull().mean() < .5]
df_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29433 entries, 0 to 29432
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   FWC           29433 non-null  float64
 1   SC_K2Q22      29396 non-null  float64
 2   K5Q10         29297 non-null  float64
 3   K7Q84_R       20976 non-null  float64
 4   TOTCSHCN      29433 non-null  float64
 5   SC_K2Q13      29394 non-null  float64
 6   K7Q04R_R      21021 non-null  float64
 7   DECISIONS     25955 non-null  float64
 8   K7Q85_R       21077 non-null  float64
 9   MENBEVCOV     27944 non-null  float64
 10  MAKEFRIEND    25599 non-null  float64
 11  BULLIED_R     21061 non-null  float64
 12  K2Q33A        29347 non-null  float64
 13  TOTNONSHCN    29433 non-null  float64
 14  SC_CSHCN      29433 non-null  float64
 15  PHYSICALPAIN  29316 non-null  float64
 16  K8Q31         29053 non-null  float64
dtypes: float64(17)
memory usage: 3.8 MB


In [20]:
# Here are the new features we should consider adding to our model
new_features_trimmed = list(df_trimmed.columns)
new_features_trimmed

['FWC',
 'SC_K2Q22',
 'K5Q10',
 'K7Q84_R',
 'TOTCSHCN',
 'SC_K2Q13',
 'K7Q04R_R',
 'DECISIONS',
 'K7Q85_R',
 'MENBEVCOV',
 'MAKEFRIEND',
 'BULLIED_R',
 'K2Q33A',
 'TOTNONSHCN',
 'SC_CSHCN',
 'PHYSICALPAIN',
 'K8Q31']

In [18]:
# We can certainly get rid of FWC, which is a statistical weight
new_features_trimmed.remove('FWC')

In [19]:
# Need to look into the codebook to see what these correspond to
new_features_trimmed

['SC_K2Q22',
 'K5Q10',
 'K7Q84_R',
 'TOTCSHCN',
 'SC_K2Q13',
 'K7Q04R_R',
 'DECISIONS',
 'K7Q85_R',
 'MENBEVCOV',
 'MAKEFRIEND',
 'BULLIED_R',
 'K2Q33A',
 'TOTNONSHCN',
 'SC_CSHCN',
 'PHYSICALPAIN',
 'K8Q31']